In [1]:
import os

In [2]:
%pwd

'/Users/ujjwaltiwari/Desktop/projects/kidney-disease-classifier-ct-scan/research'

In [3]:
os.chdir('../')

In [4]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/UjjTiw/kidney-disease-classifier-ct-scan.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'UjjTiw'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'aed5cdc8e6331b6bacc6a684372ea07f6d913399'

In [5]:
import tensorflow as tf

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int


In [7]:
from DiseaseClassifier.constants import  *
from DiseaseClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= 'artifacts/training/model.h5',
            training_data= 'artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone',
            mlflow_uri= 'https://dagshub.com/UjjTiw/kidney-disease-classifier-ct-scan.mlflow',
            all_params= self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )
    

In [11]:
import tensorflow as tf
import pathlib as Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1/255.
            validation_split= 0.30
        )

        dataflow_kwargs = dict(
            target_size= self.config.params_image_size[:-1],
            batch_size= self.config.params_batch_size,
            interpolation='bilinear'
        )

        valid_datagenerator = tf.keras.preprocessiong.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )